In [4]:
# START CREATING A SPARKSESSION OBJECT
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('data_processing').getOrCreate()
import pyspark.sql.functions as F
from pyspark.sql.types import *
#END CREATING A SPARKSESSION OBJECT
#CREATING DATAFRAMES
schema = StructType().add("user_id","string").add("country","string").add("browser","string").add("OS","string").add("age","integer")
df = spark.createDataFrame([("A203",'India',"Chrome","WIN",33),("A201",'China',"Safari","MacOS",35),("A205",'UK',"Mozilla","Linux",25)], schema=schema)
df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- browser: string (nullable = true)
 |-- OS: string (nullable = true)
 |-- age: integer (nullable = true)



In [5]:
df.show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|  India| Chrome|  WIN| 33|
|   A201|  China| Safari|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [7]:
#NULL VALUES
df_na = spark.createDataFrame([("A203",None,"Chrome","WIN",33),("A201",'China',None,"MacOS",35),("A205",'UK',"Mozilla","Linux",25)], schema=schema)
df_na.show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|   null| Chrome|  WIN| 33|
|   A201|  China|   null|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [8]:
# Fill all null value with the value 0
df_na.fillna('0').show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|      0| Chrome|  WIN| 33|
|   A201|  China|      0|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [11]:
#Replace the null values in specific columns
df_na.fillna({'country':'USA','browser':'Safari'}).show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|    USA| Chrome|  WIN| 33|
|   A201|  China| Safari|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [12]:
#Drop the rows with any null values
df_na.na.drop().show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [13]:
# Drop the row with null values for a specific column
df_na.na.drop(subset='country').show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A201|  China|   null|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [16]:
# Replace a value with another value
df_na.replace("Chrome","Google Chrome").show()

+-------+-------+-------------+-----+---+
|user_id|country|      browser|   OS|age|
+-------+-------+-------------+-----+---+
|   A203|   null|Google Chrome|  WIN| 33|
|   A201|  China|         null|MacOS| 35|
|   A205|     UK|      Mozilla|Linux| 25|
+-------+-------+-------------+-----+---+



In [18]:
# Drop a column of a dataframe
df_na.drop('user_id').show()

+-------+-------+-----+---+
|country|browser|   OS|age|
+-------+-------+-----+---+
|   null| Chrome|  WIN| 33|
|  China|   null|MacOS| 35|
|     UK|Mozilla|Linux| 25|
+-------+-------+-----+---+



In [22]:
# Create a dataframe by passing value from CSV file
df_csv=spark.read.csv("customer_data.csv",header=True,inferSchema=True)

In [23]:
# Total rows
df_csv.count()

2000

In [24]:
len(df_csv.columns)

7

In [25]:
df_csv.printSchema()

root
 |-- Customer_subtype: string (nullable = true)
 |-- Number_of_houses: integer (nullable = true)
 |-- Avg_size_household: integer (nullable = true)
 |-- Avg_age: string (nullable = true)
 |-- Customer_main_type: string (nullable = true)
 |-- Avg_Salary: integer (nullable = true)
 |-- label: integer (nullable = true)



In [26]:
# Showing three rows
df_csv.show(3)

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|Lower class large...|               1|                 3|30-40 years|Family with grown...|     44905|    0|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     37575|    0|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     27915|    0|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
only showing top 3 rows



In [27]:
df_csv.summary().show()

+-------+--------------------+------------------+------------------+-----------+--------------------+-----------------+------------------+
|summary|    Customer_subtype|  Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|       Avg_Salary|             label|
+-------+--------------------+------------------+------------------+-----------+--------------------+-----------------+------------------+
|  count|                2000|              2000|              2000|       2000|                2000|             2000|              2000|
|   mean|                null|            1.1075|            2.6895|       null|                null|     1616908.0835|            0.0605|
| stddev|                null|0.3873225521186316|0.7914562220841646|       null|                null|6822647.757312146|0.2384705099001677|
|    min|Affluent senior a...|                 1|                 1|20-30 years|      Average Family|             1361|                 0|
|    25%|                nu

In [29]:
# Select rows
df_csv.select(['Customer_subtype','Avg_Salary']).show()

+--------------------+----------+
|    Customer_subtype|Avg_Salary|
+--------------------+----------+
|Lower class large...|     44905|
|Mixed small town ...|     37575|
|Mixed small town ...|     27915|
|Modern, complete ...|     19504|
|  Large family farms|     34943|
|    Young and rising|     13064|
|Large religious f...|     29090|
|Lower class large...|      6895|
|Lower class large...|     35497|
|     Family starters|     30800|
|       Stable family|     39157|
|Modern, complete ...|     40839|
|Lower class large...|     30008|
|        Mixed rurals|     37209|
|    Young and rising|     45361|
|Lower class large...|     45650|
|Traditional families|     18982|
|Mixed apartment d...|     30093|
|Young all america...|     27097|
|Low income catholics|     23511|
+--------------------+----------+
only showing top 20 rows



In [31]:
# Filter rows and count
df_csv.filter(df_csv['Avg_Salary'] > 1000000).count()

128

In [32]:
# Filter rows and show
df_csv.filter(df_csv['Avg_Salary'] > 1000000).show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
| High status seniors|               1|                 3|40-50 years|Successful hedonists|   4670288|    0|
| High status seniors|               1|                 3|50-60 years|Successful hedonists|   9561873|    0|
| High status seniors|               1|                 2|40-50 years|Successful hedonists|  18687005|    0|
| High status seniors|               1|                 2|40-50 years|Successful hedonists|  24139960|    0|
| High status seniors|               1|                 2|50-60 years|Successful hedonists|   6718606|    0|
|High Income, expe...|               1|                 3|40-50 years|Successful hedonists|  19347139|    0|
|High Income, expe.

In [34]:
df_csv.filter(df_csv['Avg_Salary'] > 500000).filter(df_csv['Number_of_Houses'] > 2).show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    596723|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    944444|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    788477|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    994077|    0|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+



In [36]:
df_csv.where((df_csv['Avg_Salary'] > 500000) & (df_csv['Number_of_Houses'] > 2)).show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    596723|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    944444|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    788477|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    994077|    0|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+



In [37]:
# Groupe By
df_csv.groupBy('Customer_subtype').count().show()

+--------------------+-----+
|    Customer_subtype|count|
+--------------------+-----+
|Large family, emp...|   56|
|Religious elderly...|   47|
|Large religious f...|  107|
|Modern, complete ...|   93|
|    Village families|   68|
|Young all america...|   62|
|Young urban have-...|    4|
|Young seniors in ...|   22|
|Fresh masters in ...|    2|
|High Income, expe...|   52|
|Lower class large...|  288|
| Residential elderly|    6|
|Senior cosmopolitans|    1|
|        Mixed rurals|   67|
|Career and childcare|   33|
|Low income catholics|   72|
|Mixed apartment d...|   34|
|Seniors in apartm...|   17|
|Middle class fami...|  122|
|Traditional families|  129|
+--------------------+-----+
only showing top 20 rows



In [39]:
for col in df_csv.columns:
    if col != 'Avg_Salary':
        print(f"Aggregation for {col}")
        df_csv.groupBy(col).count().orderBy('count',ascending=False).show(truncate=False)

Aggregation for Customer_subtype
+------------------------------------------+-----+
|Customer_subtype                          |count|
+------------------------------------------+-----+
|Lower class large families                |288  |
|Traditional families                      |129  |
|Middle class families                     |122  |
|Large religious families                  |107  |
|Modern, complete families                 |93   |
|Couples with teens 'Married with children'|83   |
|Young and rising                          |78   |
|High status seniors                       |76   |
|Low income catholics                      |72   |
|Mixed seniors                             |71   |
|Village families                          |68   |
|Mixed rurals                              |67   |
|Young all american family                 |62   |
|Stable family                             |62   |
|Large family, employed child              |56   |
|Young, low educated                       |56   

In [40]:
df_csv.groupBy('Customer_main_type').agg(F.mean('Avg_Salary')).show()

+--------------------+--------------------+
|  Customer_main_type|     avg(Avg_Salary)|
+--------------------+--------------------+
|             Farmers|  30209.333333333332|
|       Career Loners|             32272.6|
|Retired and Relig...|   27338.80693069307|
|Successful hedonists|1.6278923510309279E7|
|         Living well|  31194.044943820223|
|      Average Family|  104256.62337662338|
|    Cruising Seniors|  28870.333333333332|
|Conservative fami...|  29504.419491525423|
|      Driven Growers|   30769.04069767442|
|Family with grown...|  28114.191881918818|
+--------------------+--------------------+



In [41]:
df_csv.groupBy('Customer_main_type').agg(F.max('Avg_Salary')).show()

+--------------------+---------------+
|  Customer_main_type|max(Avg_Salary)|
+--------------------+---------------+
|             Farmers|          49965|
|       Career Loners|          49903|
|Retired and Relig...|          49564|
|Successful hedonists|       48919896|
|         Living well|          49816|
|      Average Family|         991838|
|    Cruising Seniors|          49526|
|Conservative fami...|          49965|
|      Driven Growers|          49932|
|Family with grown...|          49901|
+--------------------+---------------+



In [42]:
df_csv.groupBy('Customer_main_type').agg(F.min('Avg_Salary')).show()

+--------------------+---------------+
|  Customer_main_type|min(Avg_Salary)|
+--------------------+---------------+
|             Farmers|          10469|
|       Career Loners|          13246|
|Retired and Relig...|           1361|
|Successful hedonists|          12705|
|         Living well|          10418|
|      Average Family|          10506|
|    Cruising Seniors|          10100|
|Conservative fami...|          10179|
|      Driven Growers|          10257|
|Family with grown...|           1502|
+--------------------+---------------+



In [43]:
df_csv.groupBy('Customer_main_type').agg(F.sum('Avg_Salary')).show()

+--------------------+---------------+
|  Customer_main_type|sum(Avg_Salary)|
+--------------------+---------------+
|             Farmers|        2809468|
|       Career Loners|         484089|
|Retired and Relig...|        5522439|
|Successful hedonists|     3158111161|
|         Living well|        5552540|
|      Average Family|       32111040|
|    Cruising Seniors|        1732220|
|Conservative fami...|        6963043|
|      Driven Growers|        5292275|
|Family with grown...|       15237892|
+--------------------+---------------+



In [44]:
df_csv.sort("Avg_Salary", ascending=False).show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
| High status seniors|               1|                 2|60-70 years|Successful hedonists|  48919896|    0|
|High Income, expe...|               1|                 2|50-60 years|Successful hedonists|  48177970|    0|
|High Income, expe...|               1|                 2|50-60 years|Successful hedonists|  48069548|    1|
|High Income, expe...|               1|                 3|40-50 years|Successful hedonists|  46911924|    0|
| High status seniors|               1|                 3|40-50 years|Successful hedonists|  46614009|    0|
|High Income, expe...|               1|                 3|30-40 years|Successful hedonists|  45952441|    0|
|High Income, expe.

In [45]:
df_csv.groupBy('Customer_subtype').agg(F.avg('Avg_Salary').alias('mean_salary')).orderBy('mean_salary',ascending=False).show(50,False)

+------------------------------------------+--------------------+
|Customer_subtype                          |mean_salary         |
+------------------------------------------+--------------------+
|High status seniors                       |2.507677857894737E7 |
|High Income, expensive child              |2.3839817807692308E7|
|Affluent young families                   |662068.7777777778   |
|Affluent senior apartments                |653638.8235294118   |
|Senior cosmopolitans                      |49903.0             |
|Students in apartments                    |35532.142857142855  |
|Large family farms                        |33135.61538461538   |
|Young, low educated                       |33072.21428571428   |
|Large family, employed child              |32867.857142857145  |
|Suburban youth                            |32558.0             |
|Village families                          |32449.470588235294  |
|Middle class families                     |31579.385245901638  |
|Modern, c

In [46]:
df_csv.groupBy('Customer_subtype').agg(F.avg('Avg_Salary').alias('max_salary')).orderBy('max_salary',ascending=False).show(50,False)

+------------------------------------------+--------------------+
|Customer_subtype                          |max_salary          |
+------------------------------------------+--------------------+
|High status seniors                       |2.507677857894737E7 |
|High Income, expensive child              |2.3839817807692308E7|
|Affluent young families                   |662068.7777777778   |
|Affluent senior apartments                |653638.8235294118   |
|Senior cosmopolitans                      |49903.0             |
|Students in apartments                    |35532.142857142855  |
|Large family farms                        |33135.61538461538   |
|Young, low educated                       |33072.21428571428   |
|Large family, employed child              |32867.857142857145  |
|Suburban youth                            |32558.0             |
|Village families                          |32449.470588235294  |
|Middle class families                     |31579.385245901638  |
|Modern, c

In [47]:
df_csv.groupBy("Customer_subtype").agg(F.collect_set("Number_of_houses")).show()

+--------------------+-----------------------------+
|    Customer_subtype|collect_set(Number_of_houses)|
+--------------------+-----------------------------+
|Large family, emp...|                       [1, 2]|
|Religious elderly...|                       [1, 2]|
|Large religious f...|                       [1, 2]|
|Modern, complete ...|                       [1, 2]|
|    Village families|                       [1, 2]|
|Young all america...|                       [1, 2]|
|Young urban have-...|                       [1, 2]|
|Young seniors in ...|                    [1, 2, 3]|
|Fresh masters in ...|                          [1]|
|High Income, expe...|                          [1]|
|Lower class large...|                       [1, 2]|
| Residential elderly|                    [1, 2, 3]|
|Senior cosmopolitans|                          [3]|
|        Mixed rurals|                          [1]|
|Career and childcare|                       [1, 2]|
|Low income catholics|                        

In [48]:
df_csv.groupBy("Customer_subtype").agg(F.collect_list("Number_of_houses")).show()

+--------------------+------------------------------+
|    Customer_subtype|collect_list(Number_of_houses)|
+--------------------+------------------------------+
|Large family, emp...|          [2, 1, 2, 1, 2, 1...|
|Religious elderly...|          [1, 1, 1, 1, 1, 1...|
|Large religious f...|          [2, 1, 1, 2, 1, 1...|
|Modern, complete ...|          [1, 1, 2, 1, 1, 1...|
|    Village families|          [1, 1, 1, 1, 1, 1...|
|Young all america...|          [1, 1, 2, 2, 1, 1...|
|Young urban have-...|                  [1, 2, 1, 1]|
|Young seniors in ...|          [1, 1, 1, 1, 1, 2...|
|Fresh masters in ...|                        [1, 1]|
|High Income, expe...|          [1, 1, 1, 1, 1, 1...|
|Lower class large...|          [1, 1, 1, 1, 1, 1...|
| Residential elderly|            [3, 1, 1, 3, 2, 1]|
|Senior cosmopolitans|                           [3]|
|        Mixed rurals|          [1, 1, 1, 1, 1, 1...|
|Career and childcare|          [2, 1, 1, 1, 1, 1...|
|Low income catholics|      

In [49]:
df_csv = df_csv.withColumn('Constant', F.lit('Finance'))
df_csv.select('Customer_subtype', 'Constant').show()

+--------------------+--------+
|    Customer_subtype|Constant|
+--------------------+--------+
|Lower class large...| Finance|
|Mixed small town ...| Finance|
|Mixed small town ...| Finance|
|Modern, complete ...| Finance|
|  Large family farms| Finance|
|    Young and rising| Finance|
|Large religious f...| Finance|
|Lower class large...| Finance|
|Lower class large...| Finance|
|     Family starters| Finance|
|       Stable family| Finance|
|Modern, complete ...| Finance|
|Lower class large...| Finance|
|        Mixed rurals| Finance|
|    Young and rising| Finance|
|Lower class large...| Finance|
|Traditional families| Finance|
|Mixed apartment d...| Finance|
|Young all america...| Finance|
|Low income catholics| Finance|
+--------------------+--------+
only showing top 20 rows



In [50]:
from pyspark.sql.functions import udf
df_csv.groupBy("Avg_age").count().show()

+-----------+-----+
|    Avg_age|count|
+-----------+-----+
|70-80 years|    8|
|50-60 years|  373|
|30-40 years|  496|
|20-30 years|   31|
|60-70 years|   64|
|40-50 years| 1028|
+-----------+-----+



In [53]:
def age_category(age):
    if age == "20-30 years":
        return "Young"
    elif age == "30-40 years":
        return "Mid aged"
    elif ((age == "40-50 years") or (age == "50-60 years")):
        return "Old"
    else:
        return "Very old"
age_udf = udf(age_category,StringType())
df_csv = df_csv.withColumn('age_category',age_udf(df_csv['Avg_age']))
df_csv.select('Avg_age','age_category').show()

+-----------+------------+
|    Avg_age|age_category|
+-----------+------------+
|30-40 years|    Mid aged|
|30-40 years|    Mid aged|
|30-40 years|    Mid aged|
|40-50 years|         Old|
|30-40 years|    Mid aged|
|20-30 years|       Young|
|30-40 years|    Mid aged|
|40-50 years|         Old|
|50-60 years|         Old|
|40-50 years|         Old|
|40-50 years|         Old|
|40-50 years|         Old|
|40-50 years|         Old|
|40-50 years|         Old|
|30-40 years|    Mid aged|
|40-50 years|         Old|
|40-50 years|         Old|
|40-50 years|         Old|
|30-40 years|    Mid aged|
|50-60 years|         Old|
+-----------+------------+
only showing top 20 rows



In [54]:
df_csv.groupBy('age_category').count().show()

+------------+-----+
|age_category|count|
+------------+-----+
|         Old| 1401|
|       Young|   31|
|    Mid aged|  496|
|    Very old|   72|
+------------+-----+



In [55]:
df_csv.select('Avg_Salary').summary().show()

+-------+-----------------+
|summary|       Avg_Salary|
+-------+-----------------+
|  count|             2000|
|   mean|     1616908.0835|
| stddev|6822647.757312146|
|    min|             1361|
|    25%|            20315|
|    50%|            31421|
|    75%|            42949|
|    max|         48919896|
+-------+-----------------+



In [58]:
min_sal = 1361
max_sal = 48919896
from pyspark.sql.functions import pandas_udf, PandasUDFType
def scaled_salary(salary):
    scaled_sal = (salary-min_sal) / (salary-max_sal)
    return scaled_sal
scaling_udf = pandas_udf(scaled_salary,DoubleType())
df_csv = df_csv.withColumn('scaled_salary',scaling_udf(df_csv['Avg_Salary'])).show(10,False)

+--------------------------+----------------+------------------+-----------+---------------------+----------+-----+--------+------------+----------------------+
|Customer_subtype          |Number_of_houses|Avg_size_household|Avg_age    |Customer_main_type   |Avg_Salary|label|Constant|age_category|scaled_salary         |
+--------------------------+----------------+------------------+-----------+---------------------+----------+-----+--------+------------+----------------------+
|Lower class large families|1               |3                 |30-40 years|Family with grown ups|44905     |0    |Finance |Mid aged    |-8.909259952600299E-4 |
|Mixed small town dwellers |1               |2                 |30-40 years|Family with grown ups|37575     |0    |Finance |Mid aged    |-7.408404359522945E-4 |
|Mixed small town dwellers |1               |2                 |30-40 years|Family with grown ups|27915     |0    |Finance |Mid aged    |-5.431156491695438E-4 |
|Modern, complete families |1     

In [61]:
region_data = spark.createDataFrame([
    ('Family with grown ups','PN'),
    ('Driven growers','CJ'),
    ('Conservative families','DD'),
    ('Cruising Seniors','DL'),
    ('Average family','ML'),
    ('Living well','KA'),
    ('Successful hedonists','JH'),
    ('Retired and Religious','AX'),
    ('Career Loners','HY'),('Farmers','JH')],
    schema=StructType().add('Customer_main_type','string').add('Region_code','string'))
region_data.show()

+--------------------+-----------+
|  Customer_main_type|Region_code|
+--------------------+-----------+
|Family with grown...|         PN|
|      Driven growers|         CJ|
|Conservative fami...|         DD|
|    Cruising Seniors|         DL|
|      Average family|         ML|
|         Living well|         KA|
|Successful hedonists|         JH|
|Retired and Relig...|         AX|
|       Career Loners|         HY|
|             Farmers|         JH|
+--------------------+-----------+



In [64]:
new_df = df_csv.join(region_data,on='Customer_main_type')
new_df.groupBy('Region_code').count().show()

+-----------+-----+
|Region_code|count|
+-----------+-----+
|         PN|  542|
|         DD|  236|
|         DL|   60|
|         KA|  178|
|         JH|  287|
|         HY|   15|
|         AX|  202|
+-----------+-----+



In [65]:
df_csv.groupBy('Customer_main_type').pivot('Avg_age').sum('Avg_Salary').fillna(0).show()

+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+
|  Customer_main_type|20-30 years|30-40 years|40-50 years|50-60 years|60-70 years|70-80 years|
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+
|             Farmers|          0|     462027|    2031235|     316206|          0|          0|
|       Career Loners|     143998|     176639|      25701|     105193|      32558|          0|
|Retired and Relig...|     126350|     336631|    2975266|    1687711|     335357|      61124|
|Successful hedonists|      42261|  171278764| 1223362814| 1563071675|  200340129|      15518|
|         Living well|     460528|    2965303|    1795405|     331304|          0|          0|
|      Average Family|          0|   23682805|    7789464|     412490|     226281|          0|
|    Cruising Seniors|          0|      43302|     303601|     529354|     716425|     139538|
|Conservative fami...|      69390|    2381485|    

In [67]:
df_csv.groupBy('Customer_main_type').pivot('label').sum('Avg_Salary').fillna(0).show()

+--------------------+----------+---------+
|  Customer_main_type|         0|        1|
+--------------------+----------+---------+
|             Farmers|   2734832|    74636|
|       Career Loners|    484089|        0|
|Retired and Relig...|   5328410|   194029|
|Successful hedonists|2720381462|437729699|
|         Living well|   5453384|    99156|
|      Average Family|  26036999|  6074041|
|    Cruising Seniors|   1675841|    56379|
|Conservative fami...|   6595027|   368016|
|      Driven Growers|   4492465|   799810|
|Family with grown...|  14394094|   843798|
+--------------------+----------+---------+



In [68]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col,row_number

win = Window.orderBy(df_csv['Avg_Salary'].desc())
df_csv = df_csv.withColumn('rank',row_number().over(win).alias('rank'))
df_csv.show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|rank|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+----+
| High status seniors|               1|                 2|60-70 years|Successful hedonists|  48919896|    0|   1|
|High Income, expe...|               1|                 2|50-60 years|Successful hedonists|  48177970|    0|   2|
|High Income, expe...|               1|                 2|50-60 years|Successful hedonists|  48069548|    1|   3|
|High Income, expe...|               1|                 3|40-50 years|Successful hedonists|  46911924|    0|   4|
| High status seniors|               1|                 3|40-50 years|Successful hedonists|  46614009|    0|   5|
|High Income, expe...|               1|                 3|30-40 years|Successful hedonis

In [69]:
win_1=Window.partitionBy('Customer_subtype').orderBy(df_csv['Avg_Salary'].desc())
df_csv = df_csv.withColumn('rank',row_number().over(win_1).alias('rank'))
df_csv.groupBy('rank').count().orderBy('rank').show()

+----+-----+
|rank|count|
+----+-----+
|   1|   39|
|   2|   37|
|   3|   36|
|   4|   36|
|   5|   34|
|   6|   34|
|   7|   32|
|   8|   31|
|   9|   31|
|  10|   31|
|  11|   31|
|  12|   31|
|  13|   31|
|  14|   31|
|  15|   31|
|  16|   30|
|  17|   30|
|  18|   27|
|  19|   27|
|  20|   27|
+----+-----+
only showing top 20 rows



In [70]:
df_csv.filter(col('rank') < 4).show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|rank|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+----+
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    994077|    0|   1|
|Affluent senior a...|               1|                 2|50-60 years|Successful hedonists|    983051|    0|   2|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    944444|    0|   3|
|Affluent young fa...|               1|                 3|30-40 years|      Average Family|    991838|    0|   1|
|Affluent young fa...|               1|                 3|40-50 years|      Average Family|    988282|    0|   2|
|Affluent young fa...|               1|                 3|30-40 years|      Average Fami